<a href="https://colab.research.google.com/github/Codewiz22/AES/blob/main/Blowfish_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding

class BlowfishCipher:
    """
    A professional-grade Blowfish implementation for academic study.
    Standard: Blowfish-CBC with PKCS7 Padding.

    Blowfish is a 64-bit block cipher designed by Bruce Schneier.
    It supports variable key lengths from 32 bits up to 448 bits.
    """
    def __init__(self, key=None):
        # Blowfish key length can range from 4 to 56 bytes (32-448 bits).
        # We default to 16 bytes (128 bits) for standard security.
        self.key = key if key else os.urandom(16)
        self.backend = default_backend()

        if not (4 <= len(self.key) <= 56):
            raise ValueError("Blowfish key must be between 4 and 56 bytes long.")

    def encrypt(self, plaintext: str) -> dict:
        """
        Encrypts plaintext using Blowfish-CBC.
        Returns a dictionary containing the ciphertext and the IV.
        """
        # Blowfish operates on 64-bit blocks, so the IV is 8 bytes.
        iv = os.urandom(8)

        # Setup Padding (Blowfish operates on 64-bit blocks)
        padder = padding.PKCS7(64).padder()
        padded_data = padder.update(plaintext.encode()) + padder.finalize()

        # Create Cipher object using Blowfish algorithm in CBC mode
        cipher = Cipher(algorithms.Blowfish(self.key),
                        modes.CBC(iv),
                        backend=self.backend)
        encryptor = cipher.encryptor()

        # Encrypt the padded data
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()

        return {
            "ciphertext": ciphertext,
            "iv": iv,
            "key_size_bits": len(self.key) * 8
        }

    def decrypt(self, ciphertext: bytes, iv: bytes) -> str:
        """
        Decrypts ciphertext using Blowfish-CBC.
        """
        cipher = Cipher(algorithms.Blowfish(self.key),
                        modes.CBC(iv),
                        backend=self.backend)
        decryptor = cipher.decryptor()

        # Decrypt the ciphertext
        padded_data = decryptor.update(ciphertext) + decryptor.finalize()

        # Remove PKCS7 Padding (64-bit block size)
        unpadder = padding.PKCS7(64).unpadder()
        data = unpadder.update(padded_data) + unpadder.finalize()

        return data.decode('utf-8')

# --- Usage Example for Week 3 Presentation ---
if __name__ == "__main__":
    print("--- Blowfish Encryption Demo ---")

    # Initialize with a 128-bit key
    bf_key = os.urandom(16)
    bf = BlowfishCipher(bf_key)

    message = "Blowfish is a fast block cipher with key-dependent S-boxes."

    # Encrypt
    result = bf.encrypt(message)
    print(f"Algorithm:       Blowfish-CBC")
    print(f"Key Size:        {result['key_size_bits']} bits")
    print(f"Original:        {message}")
    print(f"Ciphertext(hex): {result['ciphertext'].hex()}")
    print(f"IV (hex):         {result['iv'].hex()}")

    # Decrypt
    decrypted = bf.decrypt(result['ciphertext'], result['iv'])
    print(f"Decrypted:       {decrypted}")

    # Verification
    assert message == decrypted
    print("\nBlowfish Encryption Cycle: Success.")

--- Blowfish Encryption Demo ---
Algorithm:       Blowfish-CBC
Key Size:        128 bits
Original:        Blowfish is a fast block cipher with key-dependent S-boxes.
Ciphertext(hex): d75fcfe66abd43ec69fb27c9f56251ae328109f786f743bf5854522e6e5bdf5bd8b46488bf7b0f5022795d5794a331c4a2fc344c0e682294771af7e9786ee5d3
IV (hex):         fe8c214c22404d6d
Decrypted:       Blowfish is a fast block cipher with key-dependent S-boxes.

Blowfish Encryption Cycle: Success.


/tmp/ipython-input-1470385554.py:36: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  cipher = Cipher(algorithms.Blowfish(self.key),
/tmp/ipython-input-1470385554.py:54: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  cipher = Cipher(algorithms.Blowfish(self.key),
